In [ ]:
import xarray as xr

In [ ]:
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(scheduler_port=8789)
cluster

In [3]:
# from distributed import LocalCluster
# cluster = LocalCluster(n_workers=3,threads_per_worker=1)
# cluster

In [2]:
from distributed import Client
# client=Client(cluster)
client=Client('tcp://localhost:8789')
client

Client Scheduler: tcp://localhost:8789 Dashboard: http://localhost:8787/status,Cluster Workers: 40 Cores: 80 Memory: 960.00 GB


In [3]:
import zarr, os
zarr_path = '/home/jovyan/shared-data/aqua.P1D.aust.K_490.zarr'
zarr_path = os.environ['MYSCRATCH'] + '/aqua.P1D.aust.K_490.Full.zarr' # MYSCRATCH is a Pawsey HPC environment variable

In [4]:
ds = xr.open_zarr(zarr_path)
ds

<xarray.Dataset>
Dimensions:    (latitude: 7001, longitude: 10001, time: 4882)
Coordinates:
  * latitude   (latitude) float64 10.0 9.99 9.98 9.97 ... -59.98 -59.99 -60.0
  * longitude  (longitude) float64 80.0 80.01 80.02 80.03 ... 180.0 180.0 180.0
  * time       (time) datetime64[ns] 2002-07-06 2002-07-07 ... 2016-12-31
Data variables:
    K_490      (time, latitude, longitude) float32 dask.array<shape=(4882, 7001, 10001), chunksize=(7, 856, 1223)>
Attributes:
    Conventions:  CF-1.6
    history:      File initialised at 2015-12-16T15:54:44.125153\nInitialised...
    source_path:  imos-srs/archive/oc/aqua/1d/v201508/2002/07/A20020706.L2OC_...

In [ ]:
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv
import numpy as np
import cartopy
from cartopy import crs as ccrs

hv.notebook_extension('bokeh')

In [6]:
plate=ccrs.PlateCarree(central_longitude=133.5)

In [ ]:
%%opts Overlay [width=700 height=400]
%%opts Image [projection=plate]

(ds.K_490.hvplot('longitude','latitude',groupby='time',dynamic=True, rasterize=True, cmap='magma',crs=ccrs.PlateCarree()).redim(K_490=dict(range=(0, 0.5))))*\
(gv.feature.coastline.geoms('10m')).opts(color='black')


Now with zarr the performance of the IO and xarray+dask really starts to sing. The cell below doesnt calculate anything, until the geoview plot is called, then as we browse the months it calculated the mean on the fly

In [9]:
monthly_mean = ds.groupby('time.month').mean('time').compute() #or if we want to just calculate the result add .compute() to the end, however the results will be large and may kill the memory on your notebook server (needs about 9GB for all australia)
monthly_mean

<xarray.Dataset>
Dimensions:    (latitude: 7001, longitude: 10001, month: 12)
Coordinates:
  * latitude   (latitude) float64 10.0 9.99 9.98 9.97 ... -59.98 -59.99 -60.0
  * longitude  (longitude) float64 80.0 80.01 80.02 80.03 ... 180.0 180.0 180.0
  * month      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    K_490      (month, latitude, longitude) float32 nan nan nan ... 0.083 0.083

In [ ]:
%%opts Overlay [width=700 height=400]
%%opts Image [projection=plate]

(monthly_mean.hvplot('longitude','latitude',groupby='month',dynamic=True, rasterize=True, cmap='magma',crs=ccrs.PlateCarree()).redim(K_490=dict(range=(0, 0.5))))*\
(gv.feature.coastline.geoms('10m')).opts(color='black')